# Wheat detection using YOLOv5

This is a notebook for the Kaggle competition [Global Wheat Detection](https://www.kaggle.com/c/global-wheat-detection).

We use [YOLOv5](https://github.com/ultralytics/yolov5).


## Cloning repository of YOLOv5

In [1]:
!git clone https://github.com/ultralytics/yolov5
!mv yolov5/* ./

In [1]:
!python -m pip install --upgrade pip
!pip install -r requirements.txt

## Prepare training data to YOLOv5 format

More information [here](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data). We need two additional YAML files for both dataset and model configurations. The dataset configuration file is available [here](https://www.kaggle.com/viroviro/wheat-detection-yolov5-utils). The model configuration file we use is available in the repository of YOLOv5.

In [1]:
import numpy as np 
import pandas as pd 
import os
from tqdm.auto import tqdm
import shutil as sh


In [1]:
df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0

df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
df.head()

In [1]:
index = list(set(df.image_id))
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

## Train Model

We use pretrained weights on COCO dataset. The pretrained weights are auto-downloaded from [Google Drive](https://drive.google.com/drive/folders/1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J). The training can be slow, so sure you have an accelerator to GPU (1 epoch takes 15 mins approximately using GPU).

In [1]:
!python train.py --img 1024 --batch 2 --epochs 10 \
                 --data ../input/wheat-detection-yolov5-utils/wheat0.yaml \
                 --cfg models/yolov5x.yaml \
                 --weights yolov5x.pt


The final output indicates the location where the model was saved. In this case, at `runs/exp0/weights/best.pt`.

In [1]:
# copy saved model to weights folder
!cp runs/exp4/weights/best.pt weights

In [1]:
# remove convertor of training data
!rm -rf convertor

## Test Model

There are 10 test images at `../input/global-wheat-detection/test/`.

In [1]:
# Detect test images
!python detect.py --source '../input/global-wheat-detection/test/' --weight weights/best.pt --output 'inference/output' 

The output images are stored at `inference/output`.

### Display Output Images

In [1]:
!ls -l inference/output

In [1]:
from IPython.display import Image, clear_output  # to display images
Image(filename='inference/output/2fd875eaa.jpg', width=600)

In [1]:
Image(filename='inference/output/348a992bb.jpg', width=600)